In [91]:
# =============================================== usuals
import pandas as pd
import numpy as np

# =============================================== plot
import seaborn as sns
import matplotlib.pyplot as plt
# ============================================== Images
from PIL import Image
# =============================================== web scraping
import requests
import zipfile

from io import StringIO
from io import  BytesIO

from requests_html import HTMLSession 
from requests_html import AsyncHTMLSession
from datetime import datetime


from time import sleep
from tqdm import tqdm

# ================================================ warnings
import warnings
warnings.filterwarnings('ignore')

# **Web scraping**

In [92]:



def weather_series(station,years):
        session = HTMLSession() # Inicializa a classe de Requisicoes HTML
        url = 'https://portal.inmet.gov.br/dadoshistoricos'
        r = session.get(url) # Obtem as requisicoes

        # Pega todos os links de download dos arquivos com os dados climaticos historicos.
        links = [i for i in r.html.find('a') if 'https://portal.inmet.gov.br/uploads/dadoshistoricos' in str(i)]
        # <a href="https://portal.inmet.gov.br/uploads/dadoshistoricos/2001.zip">ANO 2001 (AUTOMÁTICA)</a>

        out = pd.DataFrame()
        for station in station:
            # Bloco que captura e trata os dados presentes em cada link obtido.
                for y in years:

                    url = list(filter(lambda x: str(y) in x.attrs['href'],links))[0].attrs['href'] 
                    a = BytesIO(requests.get(url).content)
                    mzip = zipfile.ZipFile(a)
                    FILE_NAME = [i.filename for i in mzip.infolist() if station in i.filename] # A001 - ESTAÇÃO DE BRASILIA
                    bytes_arq = mzip.open(name = FILE_NAME[0], mode = 'r').read()
                    string_arq = str(bytes_arq,'latin-1')
                    data = StringIO(string_arq) 
                    df_clim=pd.read_csv(data,skiprows = range(0, 8), error_bad_lines=False, encoding='latin-1',sep=';')
                    df_clim = df_clim.loc[:,~df_clim.columns.str.match("Unnamed")]


                    df_clim['Hora UTC'] = df_clim['Hora UTC'].apply(lambda x: x.replace('00 UTC',''))
                    df_clim['index'] = df_clim['Data'].astype(str)+' '+df_clim['Hora UTC'].astype(str)
                    df_clim['index'] = df_clim['index'].apply(lambda x: datetime.strptime(x,'%Y/%m/%d %H'))
                    df_clim.set_index('index',inplace=True)
                    df_clim.drop(['Data','Hora UTC'],axis=1,inplace=True)
                    df_clim = df_clim.apply(lambda x: x.apply(lambda y: str(y).replace(',','.'))) #trocar todas as virgulas por pontos
                    df_clim = df_clim.astype(float)
                
                    df_clim['CÓDIGO_ESTAÇÃO'] = station
                    out = pd.concat([out,df_clim],axis=0,ignore_index=False)
                

        New_Names=['precipitacao',
                   'pressao_instantanea',
                   'pressao_max',
                   'pressao_min',
                   'radiacao',
                   'temperatura_bulbo_seco',
                   'temperatura_orvalho_instantanea',
                   'temperatura_maxima',
                   'temperatura_minima',
                   'temperatura_maxima_orvalho',
                   'temperatura_minima_orvalho',
                   'umidade_relativa_maxima',
                   'umidade_relativa_minima',
                   'umidade_relativa_instantanea',
                    'vento_direcao', 
                    'vento_rajada',
                    'vento_velocidade', 
                    'estacao',
                    'radiacao']

        out.columns = New_Names
        return out   

In [93]:
def print_confusion_matrix(confusion_matrix,class_label,icv,fontsize=8.5):

    fig, ax = plt.subplots(figsize=(5, 5))
    try:
        heatmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cbar=False, ax=ax,cmap='Spectral')
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize,fontweight='bold')
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize,fontweight='bold')
    # axes.set_ylabel('True label',fontsize=fontsize)
    # axes.set_xlabel('Predicted label',fontsize=fontsize)
    ax.set_title(f"{class_label}",fontsize=fontsize,fontweight='bold')
    
    fig.savefig(f'//srv-viacor01/pgo/PGOC/PGOCD/Paineis/13 - Machine Learning/Turbidez/CrossValidate/Sewer/confusion_matrix/{class_label}_CV{icv}.png',dpi=fig.dpi)

In [94]:
def cross_validade(model,x,y,cv,name_model,stratify = None,multi_output=False):

        # CROSS VALIDATION - BEST MODEL
        results_cv = pd.DataFrame()
        stratify_y = stratify if multi_output else y

        for idx_cv,(train_index, test_index) in enumerate(cv.split(x,stratify_y),1):
            # ============================ split data ===================================
            X_train, y_train = x.iloc[train_index],y.iloc[train_index]
            X_test, y_test = x.iloc[test_index],y.iloc[test_index]

            # ============================ train model ===================================
            model.fit(X_train, y_train)

            # ============================ predict model ===================================
            y_hat = model.predict(X_test)
            
            # ============================ classifier reports ===================================
            c_report = classification_report(y_test, y_hat,output_dict=True)
            c_report['model'] = name_model
            c_report['CV'] = idx_cv

            results_cv = results_cv.append(pd.DataFrame(c_report))

        return results_cv

In [95]:
def balance(seq):
    from collections import Counter
    
    n = len(seq)
    classes = [(clas,float(count)) for clas,count in Counter(seq).items()]
    k = len(classes)
    
    H = -sum([ (count/n) * np.log((count/n)) for clas,count in classes]) #shannon entropy
    return H/np.log(k)

# Extração de dados

In [96]:
years = [2019,2020,2021,2022]
stations = ['A047']
df_inmet = weather_series(stations,years)
df_inmet.head(3)

,precipitacao,pressao_instantanea,pressao_max,pressao_min,radiacao,temperatura_bulbo_seco,temperatura_orvalho_instantanea,temperatura_maxima,temperatura_minima,temperatura_maxima_orvalho,temperatura_minima_orvalho,umidade_relativa_maxima,umidade_relativa_minima,umidade_relativa_instantanea,vento_direcao,vento_rajada,vento_velocidade,estacao,radiacao
index,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.8,899.2,899.2,898.6,NaN,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,A047,NaN
2019-01-01 01:00:00,1.8,900.0,900.0,899.2,NaN,19.1,18.2,19.3,19.1,18.3,18.0,95.0,92.0,94.0,275.0,3.4,0.9,A047,NaN
2019-01-01 02:00:00,0.0,900.3,900.4,900.0,NaN,18.9,18.4,19.2,18.8,18.5,18.1,97.0,94.0,97.0,237.0,2.2,0.4,A047,NaN


In [97]:
df_inmet.head(3)

,precipitacao,pressao_instantanea,pressao_max,pressao_min,radiacao,temperatura_bulbo_seco,temperatura_orvalho_instantanea,temperatura_maxima,temperatura_minima,temperatura_maxima_orvalho,temperatura_minima_orvalho,umidade_relativa_maxima,umidade_relativa_minima,umidade_relativa_instantanea,vento_direcao,vento_rajada,vento_velocidade,estacao,radiacao
index,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.8,899.2,899.2,898.6,NaN,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,A047,NaN
2019-01-01 01:00:00,1.8,900.0,900.0,899.2,NaN,19.1,18.2,19.3,19.1,18.3,18.0,95.0,92.0,94.0,275.0,3.4,0.9,A047,NaN
2019-01-01 02:00:00,0.0,900.3,900.4,900.0,NaN,18.9,18.4,19.2,18.8,18.5,18.1,97.0,94.0,97.0,237.0,2.2,0.4,A047,NaN


In [98]:
# with open("//Srv-viacor01/pgo/PGOC/PGOCD/Paineis/13 - Machine Learning/Turbidez/TABELAS/Vazoes_esgoto_tratada.bin","rb") as f:
#     df = pickle.load(f)

In [99]:
df_vz = pd.read_csv('//srv-viacor01/pgo/PGOC/PGOCD/Paineis/13 - Machine Learning/Turbidez/eebgam.csv',index_col='index')
df_vz.head(3)

,EEB.GAM.003.LIT.001.000.000
index,
2019-01-01 00:00:00,1.871891
2019-01-01 00:10:00,1.882175
2019-01-01 00:20:00,1.888610


In [100]:
df_vz['AUX'] = pd.to_datetime(df_vz.reset_index()['index'].apply(lambda x: str(x)[:-5]+'00:00').values)
idx = df_vz.index
df = df_vz.set_index('AUX').join(df_inmet,lsuffix='AUX')
df.index = pd.to_datetime(idx)
df.head(3)

,EEB.GAM.003.LIT.001.000.000,precipitacao,pressao_instantanea,pressao_max,pressao_min,radiacao,temperatura_bulbo_seco,temperatura_orvalho_instantanea,temperatura_maxima,temperatura_minima,temperatura_maxima_orvalho,temperatura_minima_orvalho,umidade_relativa_maxima,umidade_relativa_minima,umidade_relativa_instantanea,vento_direcao,vento_rajada,vento_velocidade,estacao,radiacao
index,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,1.871891,0.8,899.2,899.2,898.6,NaN,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,A047,NaN
2019-01-01 00:10:00,1.882175,0.8,899.2,899.2,898.6,NaN,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,A047,NaN
2019-01-01 00:20:00,1.888610,0.8,899.2,899.2,898.6,NaN,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,A047,NaN


# Entendimento dos dados

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29360 entries, 2019-01-01 00:00:00 to 2021-03-09 21:50:00
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   EEB.GAM.003.LIT.001.000.000      29360 non-null  float64
 1   precipitacao                     28502 non-null  float64
 2   pressao_instantanea              28502 non-null  float64
 3   pressao_max                      28484 non-null  float64
 4   pressao_min                      28484 non-null  float64
 5   radiacao                         5472 non-null   float64
 6   temperatura_bulbo_seco           28508 non-null  float64
 7   temperatura_orvalho_instantanea  27938 non-null  float64
 8   temperatura_maxima               28490 non-null  float64
 9   temperatura_minima               28490 non-null  float64
 10  temperatura_maxima_orvalho       28340 non-null  float64
 11  temperatura_minima_orvalho       28340 non-nu

In [102]:
nans_p = (df.isna().sum()/len(df))*100
pd.DataFrame(nans_p,columns=['Nulos (%)'])

,Nulos (%)
EEB.GAM.003.LIT.001.000.000,0.000000
precipitacao,2.922343
pressao_instantanea,2.922343
pressao_max,2.983651
pressao_min,2.983651
radiacao,81.362398
temperatura_bulbo_seco,2.901907
temperatura_orvalho_instantanea,4.843324
temperatura_maxima,2.963215
temperatura_minima,2.963215


In [103]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
EEB.GAM.003.LIT.001.000.000,29360.0,1.662295,0.582537,0.0,1.618106,1.766417,1.892781,3.090987
precipitacao,28502.0,0.468304,2.839530,0.0,0.000000,0.000000,0.000000,60.200000
pressao_instantanea,28502.0,898.348979,1.851491,891.4,897.100000,898.400000,899.700000,903.400000
pressao_max,28484.0,898.616402,1.822473,891.6,897.400000,898.700000,899.900000,903.600000
pressao_min,28484.0,898.080670,1.857959,891.2,896.900000,898.200000,899.400000,903.400000
radiacao,5472.0,1699.396382,1292.704506,0.0,417.900000,1594.450000,2860.525000,4142.500000
temperatura_bulbo_seco,28508.0,22.369749,3.466635,16.2,19.600000,21.300000,24.900000,32.200000
temperatura_orvalho_instantanea,27938.0,18.398468,1.594926,11.6,17.600000,18.700000,19.500000,23.200000
temperatura_maxima,28490.0,23.082984,3.714695,16.7,20.000000,22.000000,25.900000,32.500000
temperatura_minima,28490.0,21.696602,3.186485,16.1,19.300000,20.600000,23.700000,31.600000


# Preparação


In [104]:
df['estravazamento'] = df['EEB.GAM.003.LIT.001.000.000'].apply(lambda x: 1 if x > 2.7 else 0)

In [105]:
df.estravazamento.value_counts()

0    28610
1      750
Name: estravazamento, dtype: int64

In [106]:
df.drop(['radiacao'],axis=1,inplace=True)
df.drop(['estacao'],axis=1,inplace=True)
df.drop(['EEB.GAM.003.LIT.001.000.000'],axis=1,inplace=True)
df.reset_index(drop=True,inplace=True)

In [107]:
df.dropna(inplace=True)

In [108]:
df.estravazamento.value_counts()

0    27019
1      655
Name: estravazamento, dtype: int64

In [109]:
print(f'Entropia de shanon: {balance(df.estravazamento)}')

Entropia de shanon: 0.16156961815877635


# Separação

In [110]:
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [111]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score, f1_score, precision_score,confusion_matrix, classification_report

In [112]:
df

,precipitacao,pressao_instantanea,pressao_max,pressao_min,temperatura_bulbo_seco,temperatura_orvalho_instantanea,temperatura_maxima,temperatura_minima,temperatura_maxima_orvalho,temperatura_minima_orvalho,umidade_relativa_maxima,umidade_relativa_minima,umidade_relativa_instantanea,vento_direcao,vento_rajada,vento_velocidade,estravazamento
0,0.8,899.2,899.2,898.6,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,0
1,0.8,899.2,899.2,898.6,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,0
2,0.8,899.2,899.2,898.6,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,0
3,0.8,899.2,899.2,898.6,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,0
4,0.8,899.2,899.2,898.6,19.3,18.1,19.4,19.2,18.4,18.0,94.0,92.0,93.0,344.0,3.7,1.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29355,32.8,895.2,895.2,894.3,19.5,19.5,22.5,18.6,19.6,17.1,100.0,72.0,100.0,115.0,10.9,4.8,0
29356,32.8,895.2,895.2,894.3,19.5,19.5,22.5,18.6,19.6,17.1,100.0,72.0,100.0,115.0,10.9,4.8,0
29357,32.8,895.2,895.2,894.3,19.5,19.5,22.5,18.6,19.6,17.1,100.0,72.0,100.0,115.0,10.9,4.8,0
29358,32.8,895.2,895.2,894.3,19.5,19.5,22.5,18.6,19.6,17.1,100.0,72.0,100.0,115.0,10.9,4.8,0


In [113]:
dt = df.copy()
y_dt = dt.pop('estravazamento')
x_dt = dt

In [114]:
# BALANCED
scale_pos_weight = np.sqrt(df.estravazamento.value_counts()[0]/df.estravazamento.value_counts()[1])

In [115]:
# Models
skb = SelectKBest(mutual_info_classif, k=5)
ctb_sqrt = CatBoostClassifier(verbose=0,auto_class_weights = 'SqrtBalanced',loss_function='Logloss')
ctb_blcd = CatBoostClassifier(verbose=0,auto_class_weights = 'Balanced',loss_function='Logloss')
xgb = XGBClassifier(objective='binary:logistic',scale_pos_weight=scale_pos_weight)
gbc = GradientBoostingClassifier()
lgbm =  LGBMClassifier(scale_pos_weight=scale_pos_weight)

In [116]:
# Select feature
skb.fit(x_dt,y_dt)

SelectKBest(k=5,
            score_func=<function mutual_info_classif at 0x000001A6668C2940>)

In [117]:
# Models
ml_models = {'XGBClassifier':xgb,
             
             'CatBoostClassifier_SQRT':ctb_sqrt,

             'CatBoostClassifier_BLCD':ctb_blcd,

             'GradientBoostingClassifier':gbc,
             
             'LGBM' : lgbm,
            }

In [118]:
def make_df_confusion_matrix(cm,model_name):
    col = [(model_name,'medidor','1'),(model_name,'medidor','0')]
    idx = ['1','0']
    df = pd.DataFrame(cm,columns=col,index=idx)
    df.columns = pd.MultiIndex.from_tuples(df.columns)

    return df

In [119]:
results_cv = pd.DataFrame()

for model_name in tqdm(ml_models.keys()):

    # Selection feature
    try:
        features = skb.get_support()
        x  = x_dt.loc[:,features].copy()
    except:
        x = x_dt.copy()
        pass

    skf = StratifiedKFold(n_splits=3, random_state=43,shuffle=True)

    # Cross validation
    c_report = cross_validade(model = ml_models[model_name],
                              x = x,
                              y = y_dt,
                              cv = skf,
                              name_model = model_name)
    # Fit reports                        
    results_cv = results_cv.append(pd.DataFrame(c_report))

# Save results    
results_cv.to_csv(f'//srv-viacor01/pgo/PGOC/PGOCD/Paineis/13 - Machine Learning/Turbidez/CrossValidate/Swear/reports/cross_validation_sewer.txt', header=True, index=True, sep=',', mode='w')

100%|██████████| 5/5 [02:06<00:00, 25.33s/it]


In [120]:
# Results
results = pd.read_csv('//srv-viacor01/pgo/PGOC/PGOCD/Paineis/13 - Machine Learning/Turbidez/CrossValidate/Swear/reports/cross_validation_sewer.txt',index_col=[0]).reset_index()
results.drop(['CV'],axis=1,inplace=True)
results = results[results['index']!='support']
results = results.groupby(['model','index'])[results.columns].mean()
results.reset_index(inplace=True)

display(
            results[results['index']=='f1-score'].sort_values('macro avg',ascending=False).style.background_gradient(cmap='coolwarm'),
            results[results['index']=='recall'].sort_values('macro avg',ascending=False).style.background_gradient(cmap='coolwarm'),
            results[results['index']=='precision'].sort_values('macro avg',ascending=False).style.background_gradient(cmap='coolwarm'),
        )

,model,index,0,1,accuracy,macro avg,weighted avg
3,CatBoostClassifier_SQRT,f1-score,0.996217,0.856508,0.992628,0.926363,0.992911
12,XGBClassifier,f1-score,0.996067,0.853262,0.992339,0.924664,0.992688
9,LGBM,f1-score,0.995827,0.842833,0.991870,0.919330,0.992206
0,CatBoostClassifier_BLCD,f1-score,0.995396,0.832613,0.991038,0.914005,0.991543
6,GradientBoostingClassifier,f1-score,0.990120,0.321816,0.980523,0.655968,0.974303


,model,index,0,1,accuracy,macro avg,weighted avg
2,CatBoostClassifier_BLCD,recall,0.992228,0.941966,0.991038,0.967097,0.991038
14,XGBClassifier,recall,0.993597,0.940444,0.992339,0.967020,0.992339
5,CatBoostClassifier_SQRT,recall,0.994189,0.928232,0.992628,0.961211,0.992628
11,LGBM,recall,0.993597,0.920629,0.991870,0.957113,0.991870
8,GradientBoostingClassifier,recall,0.999556,0.195397,0.980523,0.597477,0.980523


,model,index,0,1,accuracy,macro avg,weighted avg
7,GradientBoostingClassifier,precision,0.980860,0.916405,0.980523,0.948633,0.979333
4,CatBoostClassifier_SQRT,precision,0.998253,0.795355,0.992628,0.896804,0.993452
13,XGBClassifier,precision,0.998550,0.781144,0.992339,0.889847,0.993405
10,LGBM,precision,0.998067,0.777617,0.991870,0.887842,0.992850
1,CatBoostClassifier_BLCD,precision,0.998585,0.746042,0.991038,0.872313,0.992607
